In [1]:
import random
from math import sin

def generate_x():
    x = []
    for _ in range(10):
        random_number = random.uniform(0, 1) - 0.5
        bit = 1 if random_number >= 0 else 0
        x.append(bit)
    return x

def initialization(population_size):
    population = [generate_x() for _ in range(population_size)]
    return population

def convert(binary_string):
    x = int(''.join(map(str, binary_string)), 2)
    return sin(x)

def tournament_selection(population):
    random_number_i = random.random()
    i = int(random_number_i * len(population))
    selected_individual_i = population[i]
    fitness_i = convert(selected_individual_i)

    random_number_j = random.random()
    j = int(random_number_j * len(population))
    selected_individual_j = population[j]
    fitness_j = convert(selected_individual_j)

    max_fitness = max(fitness_i, fitness_j)

    return selected_individual_i if max_fitness == fitness_i else selected_individual_j

def crossover(parent1, parent2, population):
    random_number = int(random.random() * 10)
    x_m = max([parent1, parent2], key=lambda x: convert(x))
    population = [ind for ind in population if ind != x_m]
    x_n = max(population, key=lambda x: convert(x))
    population.append(x_m)

    child1 = x_m[:random_number] + x_n[random_number:]
    child2 = x_n[:random_number] + x_m[random_number:]

    return child1, child2

def mutate_chromosome(population):
    random_number_i = int(random.random() * 90)  # Adjust to the current population size (90)
    chromosome_to_mutate = population[random_number_i]

    random_number_bit = int(random.random() * 10)
    mutated_chromosome = (
        chromosome_to_mutate[:random_number_bit] +
        [0 if chromosome_to_mutate[random_number_bit] == 1 else 1] +
        chromosome_to_mutate[random_number_bit + 1:]
    )

    population[random_number_i] = mutated_chromosome

def repeat_process(population_size):
    population = initialization(population_size)
    prev_top_fitness_diff = float('inf')
    generation = 0

    while True:
        parent1 = tournament_selection(population)
        parent2 = tournament_selection(population)

        child1, child2 = crossover(parent1, parent2, population)

        mutate_chromosome(population)

        population.sort(key=lambda x: convert(x), reverse=True)

        population = population[:90]  # Set the population size to 90

        # Stopping criterion
        if generation > 0:
            top_two_fitness_diff = abs(convert(population[0]) - convert(population[1]))
            if top_two_fitness_diff < 1e-6 and top_two_fitness_diff >= prev_top_fitness_diff:
                print(f"Stopping criterion met after {generation} generations.")
                break

            prev_top_fitness_diff = top_two_fitness_diff

        generation += 1

    return population

if __name__ == "__main__":
    population_size = 90

    final_population = repeat_process(population_size)

    max_fitness_chromosome = final_population[0]
    second_max_fitness_chromosome = final_population[1]

    max_fitness_value = convert(max_fitness_chromosome)
    second_max_fitness_value = convert(second_max_fitness_chromosome)

    print(f"Maximum Fitness Value: {max_fitness_value}")
    print(f"Chromosome with Max Fitness (Binary): {max_fitness_chromosome}")
    print(f"Chromosome with Max Fitness (Decimal): {int(''.join(map(str, max_fitness_chromosome)), 2)}")
    print(f"Chromosome with Max Fitness (Degrees): {int(''.join(map(str, max_fitness_chromosome)), 2)} degrees\n")

    print(f"Second Maximum Fitness Value: {second_max_fitness_value}")
    print(f"Chromosome with Second Max Fitness (Binary): {second_max_fitness_chromosome}")
    print(f"Chromosome with Second Max Fitness (Decimal): {int(''.join(map(str, second_max_fitness_chromosome)), 2)}")
    print(f"Chromosome with Second Max Fitness (Degrees): {int(''.join(map(str, second_max_fitness_chromosome)), 2)} degrees")


Stopping criterion met after 72 generations.
Maximum Fitness Value: 0.9971687571041068
Chromosome with Max Fitness (Binary): [1, 0, 0, 0, 0, 1, 1, 1, 1, 0]
Chromosome with Max Fitness (Decimal): 542
Chromosome with Max Fitness (Degrees): 542 degrees

Second Maximum Fitness Value: 0.9971687571041068
Chromosome with Second Max Fitness (Binary): [1, 0, 0, 0, 0, 1, 1, 1, 1, 0]
Chromosome with Second Max Fitness (Decimal): 542
Chromosome with Second Max Fitness (Degrees): 542 degrees


In [5]:
import numpy as np

# Generate a number between 0 to 1
def generate_x(population):
    return np.random.rand(population)

# Selection
def selection(x, population):
    c = np.round(x * population, 2)
    return np.sin(np.radians(c))

# Sort the y maximum to minimum
def sorting(y):
    return np.sort(y)[::-1]

# Rejected 20% population
#0-2pie constraint to be put
def crossover(y, population):
    selected_population_no = int(population - population * 20 / 100)
    sel_pop = y[:selected_population_no]

    # Generate random numbers
    r = np.random.rand(selected_population_no // 2)
    p1 = np.random.choice(sel_pop, selected_population_no // 2)
    p2 = np.random.choice(sel_pop, selected_population_no // 2)

    mask = (r <= 0.5)
    b = np.where(mask, (2 * r) ** (1 / 21), (2 * (1 - r)) ** (1 / 21))
    #one more condition to write 
mask=(r>0)
b=np.where(mask,(1/2*(1-r))**(1 / 21))

    ch1 = 0.5 * ((1 + b) * p1 + (1 - b) * p2)
    ch2 = 0.5 * ((1 - b) * p1 + (1 + b) * p2)

    ch = np.concatenate([ch1, ch2])
    return np.where(ch >= 0, 1, 0)

# Polynomial Mutation
def polynomial_mutation(parent, mutation_operator):
    r = np.random.rand(len(parent))
    mask = (r <= 0.5)
    d = np.where(mask, (2 * r) ** (1 / (mutation_operator + 1)) - 1, 1 - (2 * (1 - r)) ** (1 / (mutation_operator + 1)))
    mutated_child = parent + d
    return mutated_child

# Example usage
population_size = 50
x = generate_x(population_size)

for generation in range(500):
    y = selection(x, population_size)
    sorted_y = sorting(y)
    children = crossover(sorted_y, population_size)

    # Mutation
    mutated_children = polynomial_mutation(children, mutation_operator=20)

    # Evaluate fitness
    fitness_values = [np.max(y), np.max(sorted_y), np.max(children), np.max(mutated_children)]

    # Print results at the end of 500 generations
    if generation == 499:
        max_fitness_index = np.argmax(fitness_values)
        max_fitness_chromosome = mutated_children if max_fitness_index == 3 else np.eye(10)[max_fitness_index]
        max_fitness_binary = ''.join(map(str, max_fitness_chromosome.astype(int)))
        max_fitness_decimal = int(max_fitness_binary, 2)
        max_fitness_degrees = max_fitness_decimal * 360 / (2**10 - 1)

# Print the maximum fitness chromosome and its corresponding angle
print("Maximum Fitness Chromosome:", max_fitness_chromosome)

print("Maximum Fitness (Decimal):", max_fitness_decimal)
print("Maximum Fitness (Degrees):", max_fitness_degrees)


Maximum Fitness Chromosome: [1.03684528 0.89437516 0.96499289 0.98952125 1.0881538  1.00274925
 1.01459894 0.92149833 0.92405851 0.95413269 1.01761379 0.97392284
 1.05070993 1.04137316 0.99979031 0.84963249 0.8833726  1.07429249
 1.13270515 1.00839027 0.9814209  0.9074017  1.19862582 0.99377368
 1.01127833 1.02711733 1.13669339 1.00502351 0.99171639 1.09566281
 1.0264038  1.01827592 1.02129245 1.07832783 1.00316091 1.05805952
 0.98619627 1.01893147 1.10835534 1.08668136]
Maximum Fitness (Decimal): 610631088119
Maximum Fitness (Degrees): 214884840393.783
